# Yelp Data Challenge - Restaurant Recommender

Lei Chen

Nov 2018

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("ggplot")

In [2]:
df_concat = pd.read_csv('../data/df_utility.csv', encoding='ISO-8859-1')

In [3]:
df_concat.head()

,business_id,user_id,stars,cluster
0,--9e1ONYQuAa-CB_Rrw7Tw,ymlnR8UeFvB4FZL56tCZsA,5,1
1,--9e1ONYQuAa-CB_Rrw7Tw,9pSSL6X6lFpY3FCRLEH3og,5,1
2,--9e1ONYQuAa-CB_Rrw7Tw,gm8nNoA3uB4In5o_Hxpq3g,5,1
3,--9e1ONYQuAa-CB_Rrw7Tw,CEtidlXNyQzgJSdF1ubPFw,3,1
4,--9e1ONYQuAa-CB_Rrw7Tw,-Z7Nw2UF7NiBSAzfXNA_XA,5,1


In [4]:
bus_id = list(set(df_concat.business_id.values))
bus_dic = {}
i = 0
for bus in bus_id:
    bus_dic[bus] = i
    i += 1

In [2]:
df_review = pd.read_csv('../data/last_2_years_restaurant_reviews.csv', encoding='ISO-8859-1' )

In [3]:
df_review.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id,address,...,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,avg_stars,state
0,--9e1ONYQuAa-CB_Rrw7Tw,0,2017-02-14,0,VETXTwMw6qxzOVDlXfe6Tg,5,went for dinner tonight. Amazing my husband ha...,0,ymlnR8UeFvB4FZL56tCZsA,3355 Las Vegas Blvd S,...,"{'Monday': '11:30-14:0', 'Tuesday': '11:30-14:...",1.0,36.123183,-115.16919,Delmonico Steakhouse,The Strip,89109.0,1546.0,4.0,NV
1,--9e1ONYQuAa-CB_Rrw7Tw,0,2017-12-04,0,S8-8uZ7fa5YbjnEtaW15ng,5,This was an amazing dinning experience! ORDER ...,0,9pSSL6X6lFpY3FCRLEH3og,3355 Las Vegas Blvd S,...,"{'Monday': '11:30-14:0', 'Tuesday': '11:30-14:...",1.0,36.123183,-115.16919,Delmonico Steakhouse,The Strip,89109.0,1546.0,4.0,NV
2,--9e1ONYQuAa-CB_Rrw7Tw,0,2016-08-22,1,1nK5w0VNfDlnR3bOz13dJQ,5,My husband and I went there for lunch on a Sat...,1,gm8nNoA3uB4In5o_Hxpq3g,3355 Las Vegas Blvd S,...,"{'Monday': '11:30-14:0', 'Tuesday': '11:30-14:...",1.0,36.123183,-115.16919,Delmonico Steakhouse,The Strip,89109.0,1546.0,4.0,NV
3,--9e1ONYQuAa-CB_Rrw7Tw,0,2016-09-13,0,N1Z93BthdJ7FT2p5S22jIA,3,Went for a nice anniversary dinner. Researched...,0,CEtidlXNyQzgJSdF1ubPFw,3355 Las Vegas Blvd S,...,"{'Monday': '11:30-14:0', 'Tuesday': '11:30-14:...",1.0,36.123183,-115.16919,Delmonico Steakhouse,The Strip,89109.0,1546.0,4.0,NV
4,--9e1ONYQuAa-CB_Rrw7Tw,0,2015-02-02,0,_Uwp6FO1X-avE9wqTMC59w,5,This place is first class in every way. Lobste...,0,-Z7Nw2UF7NiBSAzfXNA_XA,3355 Las Vegas Blvd S,...,"{'Monday': '11:30-14:0', 'Tuesday': '11:30-14:...",1.0,36.123183,-115.16919,Delmonico Steakhouse,The Strip,89109.0,1546.0,4.0,NV


### Important metrics: df_res

In [4]:
df_res = pd.read_csv('../data/df_res.csv')
df_res.head(5)

,address,attributes,business_id,categories,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,cluster
0,2255 N Rampart Blvd,"{'Alcohol': 'none', 'Ambience': ""{'romantic': ...",kgffcoxT6BQp-gJ-UQ7Czw,"Fast Food, Restaurants, Sandwiches","{'Monday': '9:0-21:0', 'Tuesday': '9:0-21:0', ...",1,36.201794,-115.281981,Subway,Summerlin,89128.0,13,2.5,0
1,2227 N Rampart Blvd,"{'Alcohol': 'beer_and_wine', 'Ambience': ""{'ro...",0jtRI7hVMpQHpUVtUy4ITw,"Beer, Wine & Spirits, Italian, Food, American ...","{'Monday': '7:0-14:30', 'Tuesday': '7:0-19:0',...",1,36.201990,-115.283122,Omelet House Summerlin,Summerlin,89128.0,242,4.0,4
2,4505 E Bonanza Rd,"{'Alcohol': 'none', 'Ambience': ""{'romantic': ...",zhxnD7J5_sCrKSw5cwI9dQ,"Chicken Wings, Restaurants, Fast Food","{'Monday': '10:0-23:0', 'Tuesday': '10:0-23:0'...",1,36.173140,-115.077945,Popeyes Louisiana Kitchen,Sunrise,89110.0,16,1.5,8
3,"560 N. Nellis Blvd, Ste E1","{'Alcohol': 'none', 'Ambience': ""{'romantic': ...",2kWrSFkIes_d2BMg4YrRtA,"Restaurants, Pizza","{'Monday': '11:0-23:0', 'Tuesday': '11:0-23:0'...",1,36.169353,-115.061694,Pizza Hut,Sunrise,89110.0,19,2.5,5
4,3480 S Maryland Pkwy,"{'BusinessAcceptsCreditCards': 'True', 'GoodFo...",6llKs7K_tn8ChXcIM-oTvg,"Japanese, Restaurants",NaN,1,36.125934,-115.135253,Sansei Japan,Eastside,89169.0,3,4.5,3


In [6]:
df_temp = df_res[['business_id', 'cluster']]

In [7]:
df_temp.head(5)

,business_id,cluster
0,kgffcoxT6BQp-gJ-UQ7Czw,0
1,0jtRI7hVMpQHpUVtUy4ITw,4
2,zhxnD7J5_sCrKSw5cwI9dQ,8
3,2kWrSFkIes_d2BMg4YrRtA,5
4,6llKs7K_tn8ChXcIM-oTvg,3


In [18]:
df_concat = pd.merge(df_review, df_temp, on='business_id')

In [9]:
df_concat.head(5)

,business_id,cool,date,funny,review_id,stars,text,useful,user_id,address,...,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,avg_stars,state,cluster
0,--9e1ONYQuAa-CB_Rrw7Tw,0,2017-02-14,0,VETXTwMw6qxzOVDlXfe6Tg,5,went for dinner tonight. Amazing my husband ha...,0,ymlnR8UeFvB4FZL56tCZsA,3355 Las Vegas Blvd S,...,1.0,36.123183,-115.16919,Delmonico Steakhouse,The Strip,89109.0,1546.0,4.0,NV,1
1,--9e1ONYQuAa-CB_Rrw7Tw,0,2017-12-04,0,S8-8uZ7fa5YbjnEtaW15ng,5,This was an amazing dinning experience! ORDER ...,0,9pSSL6X6lFpY3FCRLEH3og,3355 Las Vegas Blvd S,...,1.0,36.123183,-115.16919,Delmonico Steakhouse,The Strip,89109.0,1546.0,4.0,NV,1
2,--9e1ONYQuAa-CB_Rrw7Tw,0,2016-08-22,1,1nK5w0VNfDlnR3bOz13dJQ,5,My husband and I went there for lunch on a Sat...,1,gm8nNoA3uB4In5o_Hxpq3g,3355 Las Vegas Blvd S,...,1.0,36.123183,-115.16919,Delmonico Steakhouse,The Strip,89109.0,1546.0,4.0,NV,1
3,--9e1ONYQuAa-CB_Rrw7Tw,0,2016-09-13,0,N1Z93BthdJ7FT2p5S22jIA,3,Went for a nice anniversary dinner. Researched...,0,CEtidlXNyQzgJSdF1ubPFw,3355 Las Vegas Blvd S,...,1.0,36.123183,-115.16919,Delmonico Steakhouse,The Strip,89109.0,1546.0,4.0,NV,1
4,--9e1ONYQuAa-CB_Rrw7Tw,0,2015-02-02,0,_Uwp6FO1X-avE9wqTMC59w,5,This place is first class in every way. Lobste...,0,-Z7Nw2UF7NiBSAzfXNA_XA,3355 Las Vegas Blvd S,...,1.0,36.123183,-115.16919,Delmonico Steakhouse,The Strip,89109.0,1546.0,4.0,NV,1


In [9]:
df_concat.set_index('user_id').head()

,business_id,cool,date,funny,review_id,stars,text,useful,address,attributes,...,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,avg_stars,state,cluster
user_id,,,,,,,,,,,,,,,,,,,,,
ymlnR8UeFvB4FZL56tCZsA,--9e1ONYQuAa-CB_Rrw7Tw,0,2017-02-14,0,VETXTwMw6qxzOVDlXfe6Tg,5,went for dinner tonight. Amazing my husband ha...,0,3355 Las Vegas Blvd S,"{'Alcohol': 'full_bar', 'Ambience': ""{'romanti...",...,1.0,36.123183,-115.16919,Delmonico Steakhouse,The Strip,89109.0,1546.0,4.0,NV,1
9pSSL6X6lFpY3FCRLEH3og,--9e1ONYQuAa-CB_Rrw7Tw,0,2017-12-04,0,S8-8uZ7fa5YbjnEtaW15ng,5,This was an amazing dinning experience! ORDER ...,0,3355 Las Vegas Blvd S,"{'Alcohol': 'full_bar', 'Ambience': ""{'romanti...",...,1.0,36.123183,-115.16919,Delmonico Steakhouse,The Strip,89109.0,1546.0,4.0,NV,1
gm8nNoA3uB4In5o_Hxpq3g,--9e1ONYQuAa-CB_Rrw7Tw,0,2016-08-22,1,1nK5w0VNfDlnR3bOz13dJQ,5,My husband and I went there for lunch on a Sat...,1,3355 Las Vegas Blvd S,"{'Alcohol': 'full_bar', 'Ambience': ""{'romanti...",...,1.0,36.123183,-115.16919,Delmonico Steakhouse,The Strip,89109.0,1546.0,4.0,NV,1
CEtidlXNyQzgJSdF1ubPFw,--9e1ONYQuAa-CB_Rrw7Tw,0,2016-09-13,0,N1Z93BthdJ7FT2p5S22jIA,3,Went for a nice anniversary dinner. Researched...,0,3355 Las Vegas Blvd S,"{'Alcohol': 'full_bar', 'Ambience': ""{'romanti...",...,1.0,36.123183,-115.16919,Delmonico Steakhouse,The Strip,89109.0,1546.0,4.0,NV,1
-Z7Nw2UF7NiBSAzfXNA_XA,--9e1ONYQuAa-CB_Rrw7Tw,0,2015-02-02,0,_Uwp6FO1X-avE9wqTMC59w,5,This place is first class in every way. Lobste...,0,3355 Las Vegas Blvd S,"{'Alcohol': 'full_bar', 'Ambience': ""{'romanti...",...,1.0,36.123183,-115.16919,Delmonico Steakhouse,The Strip,89109.0,1546.0,4.0,NV,1


In [5]:
df_user = df_concat.set_index('user_id')['cluster']
df_user.head()

user_id
ymlnR8UeFvB4FZL56tCZsA    1
9pSSL6X6lFpY3FCRLEH3og    1
gm8nNoA3uB4In5o_Hxpq3g    1
CEtidlXNyQzgJSdF1ubPFw    1
-Z7Nw2UF7NiBSAzfXNA_XA    1
Name: cluster, dtype: int64

In [6]:
df_user = pd.get_dummies(df_user, prefix='cluster_')

In [7]:
df_user.head()

,cluster__0,cluster__1,cluster__2,cluster__3,cluster__4,cluster__5,cluster__6,cluster__7,cluster__8,cluster__9,cluster__10,cluster__11
user_id,,,,,,,,,,,,
ymlnR8UeFvB4FZL56tCZsA,0,1,0,0,0,0,0,0,0,0,0,0
9pSSL6X6lFpY3FCRLEH3og,0,1,0,0,0,0,0,0,0,0,0,0
gm8nNoA3uB4In5o_Hxpq3g,0,1,0,0,0,0,0,0,0,0,0,0
CEtidlXNyQzgJSdF1ubPFw,0,1,0,0,0,0,0,0,0,0,0,0
-Z7Nw2UF7NiBSAzfXNA_XA,0,1,0,0,0,0,0,0,0,0,0,0


In [8]:
df_user = df_user.groupby(level=0).sum()

In [9]:
df_user.shape

(260210, 12)

In [10]:
len(df_concat['user_id'].unique())

260210

In [11]:
df_user.head()

,cluster__0,cluster__1,cluster__2,cluster__3,cluster__4,cluster__5,cluster__6,cluster__7,cluster__8,cluster__9,cluster__10,cluster__11
user_id,,,,,,,,,,,,
---1lKK3aKOuomHnwAkAow,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
---udAKDsn0yQXmzbWQNSw,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
--0sXNBv6IizZXuV-nl0Aw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
--2bpE5vyR-2hAP7sZZ4lA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
--2vR0DIsmQ6WfcSzKWigw,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [13]:
df_user_idx = np.argsort(df_user, axis=1)
df_user_idx.head()

,cluster__0,cluster__1,cluster__2,cluster__3,cluster__4,cluster__5,cluster__6,cluster__7,cluster__8,cluster__9,cluster__10,cluster__11
user_id,,,,,,,,,,,,
---1lKK3aKOuomHnwAkAow,0,2,4,5,6,7,8,9,11,3,1,10
---udAKDsn0yQXmzbWQNSw,0,2,3,4,5,6,7,8,9,10,11,1
--0sXNBv6IizZXuV-nl0Aw,0,1,2,3,4,5,6,7,8,9,10,11
--2bpE5vyR-2hAP7sZZ4lA,0,1,2,3,4,5,6,7,8,9,11,10
--2vR0DIsmQ6WfcSzKWigw,0,2,3,4,5,7,8,10,11,6,9,1


In [14]:
list(df_user_idx)

['cluster__0',
 'cluster__1',
 'cluster__2',
 'cluster__3',
 'cluster__4',
 'cluster__5',
 'cluster__6',
 'cluster__7',
 'cluster__8',
 'cluster__9',
 'cluster__10',
 'cluster__11']

In [15]:
df_user['most_cluster'] = df_user_idx['cluster__11']

In [74]:
df_user.head()

,cluster__0,cluster__1,cluster__2,cluster__3,cluster__4,cluster__5,cluster__6,cluster__7,cluster__8,cluster__9,cluster__10,cluster__11,most_cluster
user_id,,,,,,,,,,,,,
---1lKK3aKOuomHnwAkAow,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,10
---udAKDsn0yQXmzbWQNSw,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
--0sXNBv6IizZXuV-nl0Aw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,11
--2bpE5vyR-2hAP7sZZ4lA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,10
--2vR0DIsmQ6WfcSzKWigw,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1


In [16]:
df_user_most_cluster = df_user[['most_cluster']]
df_user_most_cluster.head()

,most_cluster
user_id,
---1lKK3aKOuomHnwAkAow,10
---udAKDsn0yQXmzbWQNSw,1
--0sXNBv6IizZXuV-nl0Aw,11
--2bpE5vyR-2hAP7sZZ4lA,10
--2vR0DIsmQ6WfcSzKWigw,1


In [17]:
df_user_most_csv = '../data/df_user_most.csv'
df_user_most_cluster.to_csv(df_user_most_csv, index=True)

#### Start

In [26]:
df_res_cluster = [df_concat[df_concat['cluster'] == i] for i in range(12)]

In [27]:
df_res_cluster[0].head(5)

,business_id,user_id,stars,cluster
2571,-LMycE26AortJDsbc8oXOg,MrgrBZBe9Fix0oGncpYTAQ,2,0
2572,-LMycE26AortJDsbc8oXOg,LTUFysTYpEX25sCUy-ORtg,1,0
2573,-LMycE26AortJDsbc8oXOg,JaPYkrKv8Z5hkSc237Ss9Q,4,0
2574,-LMycE26AortJDsbc8oXOg,UDKqdVT0FrpL19shSOqgow,5,0
2575,-LMycE26AortJDsbc8oXOg,1_8CkXqRdtR3oMk8JzSStw,1,0


## 1. Clean data and get rating data 

#### Select relevant columns in the original dataframe

In [28]:
df_user_most = pd.read_csv('../data/df_user_most.csv')

In [29]:
del df_concat

In [30]:
# Get business_id, user_id, stars for recommender
df_rec = []
for df_tmp in df_res_cluster:
    df_rec.append(df_tmp[['business_id', 'user_id', 'stars']])
df_rec[0].head()

,business_id,user_id,stars
2571,-LMycE26AortJDsbc8oXOg,MrgrBZBe9Fix0oGncpYTAQ,2
2572,-LMycE26AortJDsbc8oXOg,LTUFysTYpEX25sCUy-ORtg,1
2573,-LMycE26AortJDsbc8oXOg,JaPYkrKv8Z5hkSc237Ss9Q,4
2574,-LMycE26AortJDsbc8oXOg,UDKqdVT0FrpL19shSOqgow,5
2575,-LMycE26AortJDsbc8oXOg,1_8CkXqRdtR3oMk8JzSStw,1


#### Create utility matrix from records

In [31]:
bus_id[0]

'bFvtf5LD-yaH7Z3RQcH7FA'

In [32]:
bus_dic['bFvtf5LD-yaH7Z3RQcH7FA']

0

In [33]:
df_utility = []

for df_tmp in df_rec:
    df_utility.append(pd.pivot_table(data = df_tmp,
                                    values='stars',
                                    index='user_id',
                                    columns='business_id',
                                    fill_value=0))

## 2. Item-Item similarity recommender

### Calculate item-item similarity matrix

In [34]:
from sklearn.metrics.pairwise import cosine_similarity
item_sim_mat = []
for df_ut in df_utility:
    item_sim_mat.append(cosine_similarity(df_ut.T))

In [35]:
item_sim_mat[0].shape

(269, 269)

### Calculate neighborhood

In [36]:
least_to_most_sim_indexes = []
for item_mat in item_sim_mat:
    least_to_most_sim_indexes.append(np.argsort(item_mat, axis=1))
least_to_most_sim_indexes[0].shape

(269, 269)

In [37]:
# Get neighborhoods
neighborhood_size = 10
neighborhoods = []
for least_index in least_to_most_sim_indexes:
    neighborhoods.append(least_index[:, -neighborhood_size:])
neighborhoods[0].shape

(269, 10)

### Make rating prediction on a user

In [38]:
user_id = np.random.choice(df_user_most['user_id'], 1)
random_user = user_id[0]

In [39]:
random_user

'BBlqKgbs_iDV-rH6-mdzjQ'

In [41]:
total = df_user.loc[random_user].sum()
user_like = df_user.loc[random_user].map(lambda x: x / total)
user_like = user_like[user_like > 0]

In [42]:
user_like

cluster__11    1.0
Name: BBlqKgbs_iDV-rH6-mdzjQ, dtype: float64

In [61]:
cluster_name = df_user.columns.values.tolist()

In [56]:
user_like_cluster = user_like.index.values.tolist()
user_like_perc = user_like.values.tolist()

In [62]:
def cluster_to_num(lst):
    ans = []
    for elem in lst:
        for i in range(len(cluster_name)):
            if(elem == cluster_name[i]):
                ans.append(i)
    return ans

In [63]:
cluster_num = cluster_to_num(user_like_cluster)

In [64]:
cluster_num

[11]

In [66]:
df_utl = list(df_utility[cluster_num[0]].columns.values)
df_utl

['0-cZ9XfpC3vsLotOJ6Nxfg',
 '0InA3Ffj8MxyQV9hgsyYcQ',
 '0NmTwqYEQiKErDv4a55obg',
 '0mnnlFCploePlnEaddw63g',
 '0qet57CmMA5qUm6gPFUTpg',
 '12P7GjIoq6oj7qCyJ_c0Ag',
 '1WBVqmo7tgF0z0eGf4fPhQ',
 '21sGRVR7HEs_t6PdB9tGMw',
 '2O_b-IAAW7SoPt__RX1Kww',
 '2bK1ng2mnRdtTh_BDNGmIg',
 '36ckWelVitpgX16g7lecqA',
 '45513UgSlMqWV_yAPVkXrg',
 '4EfiNcKSsafogIhAjwv0gg',
 '4KmrrhtfnngTVFa2d5LAjg',
 '4VkLiFoIEjTTpbCRgxjnCQ',
 '4yJVSK-Q5UkED6RjORRssA',
 '5CJOaZc8Muu8WothauxQ4g',
 '5gv6AqHXfi3gJV4fb432Vw',
 '5keoVVM81g-nJw0Zdpl1nA',
 '60uVlCUiLJvq3xNo_3bhQQ',
 '64o_nYvzIKwHFOSYSEm-qw',
 '6H8xfhoZ2IGa3eNiY5FqLA',
 '6TQ0gCdAqaBbuDyCGa0joA',
 '6jDD-Z8QcsKTdIDWwM8gog',
 '7NucoWCjPzZyRPjsVR28xQ',
 '82-wE5pkxyBKhfxyyZap_A',
 '8dSIAoz_UphMGpZr_uMing',
 '8olG8J91NG_QU92WHG5QfA',
 '8yiLVWzJyyTEhYE0fVNvlA',
 'AN0bWhisCf6LN9eHZ7DQ3w',
 'AZlnpvILz5cEWJifjr2CSQ',
 'Ay34xZOFax3X23zaA3nwdg',
 'BOQOuwSNhfKx1xvKtONsBg',
 'CSaVK9wgqe2_tQuuCDINyQ',
 'CvpXZRF-I7wfm19dW1jOXA',
 'DGDBFpZk-NIvah4M7MtDDg',
 'DLJCwHGtbAV31UQ1lnIjlw',
 

In [67]:
def item_list_to_id(item_list):
    ans = []
    for item in item_list:
        ans.append(df_utl[item])
    return ans

In [70]:
import time


pred_lst = []
for user_cluster in cluster_num:
    n_users = df_utility[user_cluster].shape[0]
    n_items = df_utility[user_cluster].shape[1]
    start_time = time.time()
    items_rated_by_this_user = df_utility[user_cluster].loc[random_user].nonzero()[0]
    # Just initializing so we have somewhere to put rating preds
    out = np.zeros(n_items)
    for item_to_rate in range(n_items):
        relevant_items = np.intersect1d(neighborhoods[user_cluster][item_to_rate],
                                        items_rated_by_this_user,
                                        assume_unique=True)  # assume_unique speeds up intersection op
        out[item_to_rate] = (df_utility[user_cluster].loc[random_user, item_list_to_id(relevant_items)] * \
            item_sim_mat[user_cluster][item_to_rate, relevant_items] / \
            item_sim_mat[user_cluster][item_to_rate, relevant_items]).sum()


    pred_ratings = np.nan_to_num(out)
    pred_lst.append(pred_ratings)
    print(pred_ratings)
    print("Execution time: %f seconds" % (time.time()-start_time))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 5. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 5. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 5. 0. 0. 0. 0. 5. 0. 0. 0. 0. 0. 0. 5. 0. 0.
 0. 0. 5. 0. 0. 0. 0. 5. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 5. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 5. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 5.]
Execution time: 0.277438 seconds


/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  if __name__ == '__main__':


In [71]:
pred_lst[0].shape

(146,)

In [85]:
import pickle

with open('../data/business.pkl', 'rb') as f:
    bus_name = pickle.load(f)

bus_name

{'--9e1ONYQuAa-CB_Rrw7Tw': 'Delmonico Steakhouse',
 '-1m9o3vGRA8IBPNvNqKLmA': "Bavette's Steakhouse & Bar",
 '-1vfRrlnNnNJ5boOVghMPA': 'Red Ginseng Narita Sushi & BBQ',
 '-3zffZUHoY8bQjGfPSoBKQ': 'Michael Mina',
 '-8R_-EkGpUhBk55K9Dd4mg': 'Sin City Thai Restaurant',
 '-9YyInW1wapzdNZrhQJ9dg': 'Fresh Buffet',
 '-AD5PiuJHgdUcAK-Vxao2A': 'Smashburger',
 '-ADtl9bLp8wNqYX1k3KuxA': 'Ichi Ramen House',
 '-AGdGGCeTS-njB_8GkUmjQ': 'Woo Chun Korean BBQ',
 '-BS4aZAQm9u41YnB9MUASA': 'Argana',
 '-Bf8BQ3yMk8U2f45r2DRKw': 'In-N-Out Burger',
 '-BmqghX1sv7sgsxOIS2yAg': 'Merkato Ethiopian Cafe',
 '-Bv-HHUs8aHzDrdWcZHn8w': 'Smashburger',
 '-C8sSrFqaCxp51pyo-fQLQ': 'The Rice Shop',
 '-CQokjildrY7UZezXCdEBw': 'Toddy Shop',
 '-CbDQXiuKzPQ0_jiUz03aw': 'Taiga Modern Japanese & Thai Restaurant',
 '-EohS3nXZGl6odE9Lx5BSw': 'Panda Express',
 '-FNquqGseSCVMWo7KbK-Tg': 'Chipotle Mexican Grill',
 '-FcZY7a7qgxTUlTvwuyJnQ': '8 Noodle Bar',
 '-G7MPSNBpxRJmtrJxdwt7A': 'La Costa Del Sol',
 '-IWsoxH7mLJTTpU5MmWY4w': 'Pho

In [86]:
# Recommend n restaruant

for i in range(len(cluster_num)):
    print("Cluster %d" % i)
    n = int(10 / user_like_perc[i])

    # Get item indexes sorted by predicted rating
    item_index_sorted_by_pred_rating = list(np.argsort(pred_lst[i]))[::-1]

    # Find items that have been rated by user
    items_rated_by_this_user = df_utility[cluster_num[i]].loc[random_user].nonzero()[0]

    # We want to exclude the items that have been rated by user
    unrated_items_by_pred_rating = [item for item in item_index_sorted_by_pred_rating
                                    if item not in items_rated_by_this_user]

    unrated_items_by_pred_rating[:n]
    
    for num in unrated_items_by_pred_rating[:n]:
        print(bus_name[df_utl[num]])

Cluster 0
Magura
Spaghetty Western
Carrabba's Italian Grill
Villa Pizza Durango
Russo's Pizza Kitchen
Tuscany Gardens
Roma Deli 1 & Restaurant
Giordanos - Las Vegas
Cugino's Italian Deli & Pizza
North End Pizza


/opt/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  # This is added back by InteractiveShellApp.init_path()


In [40]:
for num in unrated_items_by_pred_rating[:n]:
    print(bus_name[df_utl[num]])

North End Pizza
My Mother's House Italian Restaurant
North Beach Italian Bistro
Los Olivos Ristorante
Sbarro
Cafe Modena
The Tap House
Joe Philly's Pizza
Casanova
Capo's Italian Cuisine
